In [1]:
import pymysql
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
def create_dummies(dataset,columns):
    
    for column in columns:
        dummy=pd.get_dummies(dataset[column],prefix=column)

        dataset=dataset.drop(column,axis=1)#axis=1-->para eliminar columna y no fila

        dataset=pd.concat([dataset,dummy],axis=1)#axis=1-->para agregar columna y no fila

    return dataset

In [3]:
def normalize_columns(dataset,columns):
    scaler = MinMaxScaler()
    scaled_X = scaler.fit_transform(dataset[columns])
    X_normalized=pd.DataFrame(scaled_X,columns=columns)
    dataset=dataset.drop(columns,axis=1)#axis=1-->para eliminar columna y no fila
    new_dataset=pd.concat([dataset,X_normalized],axis=1)#axis=1-->para agregar columna y no fila
    return new_dataset

In [4]:
try:
    conn=pymysql.connect(host='localhost', user='root', passwd='', db='unifin')
    cur=conn.cursor()
    query="SELECT Id_cliente,Macro_sector,Sector,Subsector,Actividad,Ventas,Activo_fijo,Potencial,Cheques,Etapa,Producto from nbo_model;"
    cur.execute(query)
    res = cur.fetchall()
    cur.close()
    conn.close()
    
except pymysql.Error as e:
    msj= ("Error %d: %s" % (e.args[0], e.args[1]))
    print(msj)
else:
    headers=['id','Macro_sector','Sector','Subsector','Actividad','Ventas','Activo_fijo','Potencial','Cheques','Etapa','Producto']
    dataset_dummy={}
    filas=0

    for h in headers:
        dataset_dummy[h]=[]

    if(len(res)>0):
        for r in res:
            filas+=1
            for i in range(len(headers)):
                dataset_dummy[headers[i]].append(r[i])

    print('El numero de filas de este dataset es de:'+str(filas))

    df1=pd.DataFrame(dataset_dummy) 
    print(df1.shape)

    print(df1.dtypes)

El numero de filas de este dataset es de:30069
(30069, 11)
id               object
Macro_sector     object
Sector           object
Subsector        object
Actividad        object
Ventas          float64
Activo_fijo     float64
Potencial       float64
Cheques          object
Etapa            object
Producto         object
dtype: object


## Analisis de datos

In [5]:
#df2=create_dummies(df1,['Etapa'])

In [6]:
yfails=df1[(df1['Producto']=='function(){return th') | (df1['Producto']=='')].index.tolist()

df1=df1.drop(yfails)
df1=df1.reset_index(drop=True)

df1.shape

(30059, 11)

In [7]:
Y=df1['Producto']
X=df1[['Macro_sector','Sector','Subsector','Actividad','Ventas','Activo_fijo','Potencial']]
X.shape

(30059, 7)

In [8]:
#X['Cheques']=X['Cheques'].fillna(0)
indexes_empties=X[pd.isnull(X).any(axis=1)].index.tolist()

X=X.dropna(axis=0,how="any")
X= X.reset_index(drop=True)

Y=Y.drop(indexes_empties)
Y= Y.reset_index(drop=True)

X.shape

(29991, 7)

In [9]:
Y.unique()

array(['2', '1', '3', '4', '5'], dtype=object)

In [10]:
X_normalized=normalize_columns(X,['Ventas','Activo_fijo','Potencial'])
X_normalized.shape

(29991, 7)

In [11]:
X_normalized=create_dummies(X_normalized,['Macro_sector','Sector','Subsector','Actividad'])
X_normalized.shape

(29991, 1020)

In [12]:
X_normalized=X_normalized[['Activo_fijo', 'Macro_sector_', 'Macro_sector_1', 'Macro_sector_2', 'Macro_sector_3', 'Macro_sector_4', 'Macro_sector_5', 'Sector_', 'Sector_1300003', 'Sector_3400009', 'Sector_4100004', 'Sector_6700000', 'Sector_6800002', 'Sector_6900006', 'Sector_7300007', 'Sector_9999999', 'Subsector_', 'Subsector_00000000', 'Subsector_00010000', 'Subsector_0200006-2', 'Subsector_1400001-1', 'Subsector_2300002-1', 'Subsector_2900000-2', 'Subsector_3000007-2', 'Subsector_3400009-2', 'Subsector_3500007-2', 'Subsector_3900009-11', 'Subsector_3900009-6', 'Subsector_6200000-3', 'Subsector_6600002-2', 'Subsector_6600002-5', 'Subsector_6600002-6', 'Subsector_6600002-7', 'Subsector_6800001-1', 'Subsector_6900006-1', 'Subsector_7100001-3', 'Subsector_8100000-3', 'Subsector_8100000-5', 'Subsector_8200008-1', 'Subsector_8200008-4', 'Subsector_8300006-1', 'Subsector_8400004-1', 'Subsector_8400004-2', 'Subsector_8400004-3', 'Subsector_8900004-2', 'Subsector_8900004-5', 'Subsector_9400003-1', 'Subsector_9999999-1', 'Actividad_', 'Actividad_0111063', 'Actividad_0112095', 'Actividad_0112128', 'Actividad_0232017', 'Actividad_0291013', 'Actividad_0312017', 'Actividad_1321017', 'Actividad_1411016', 'Actividad_2012037', 'Actividad_2025014', 'Actividad_2028026', 'Actividad_2049030', 'Actividad_2059013', 'Actividad_2061026', 'Actividad_2094019', 'Actividad_2096015', 'Actividad_2312031', 'Actividad_2321016', 'Actividad_2329010', 'Actividad_2394021', 'Actividad_2411015', 'Actividad_2413011', 'Actividad_2429018', 'Actividad_2431013', 'Actividad_2512011', 'Actividad_2529107', 'Actividad_2632017', 'Actividad_2912005', 'Actividad_3032018', 'Actividad_3092046', 'Actividad_3097054', 'Actividad_3112018', 'Actividad_3113016', 'Actividad_3122017', 'Actividad_3229045', 'Actividad_3323029', 'Actividad_3411022', 'Actividad_3421013', 'Actividad_3516038', 'Actividad_3532026', 'Actividad_3591022', 'Actividad_3596014', 'Actividad_3599018', 'Actividad_3694024', 'Actividad_3697010', 'Actividad_3723013', 'Actividad_3799014', 'Actividad_3812014', 'Actividad_3961027', 'Actividad_3999119', 'Actividad_4111027', 'Actividad_4111910', 'Actividad_4112017', 'Actividad_4121018', 'Actividad_4121026', 'Actividad_4121034', 'Actividad_4123014', 'Actividad_4129012', 'Actividad_4193017', 'Actividad_4222014', 'Actividad_4291019', 'Actividad_6135017', 'Actividad_6225024', 'Actividad_6311039', 'Actividad_6321012', 'Actividad_6329016', 'Actividad_6512017', 'Actividad_6514013', 'Actividad_6623020', 'Actividad_6625018', 'Actividad_6695011', 'Actividad_6699021', 'Actividad_6699039', 'Actividad_6711015', 'Actividad_6712013', 'Actividad_6714019', 'Actividad_6714027', 'Actividad_6732029', 'Actividad_6800002', 'Actividad_6811013', 'Actividad_6813027', 'Actividad_6814017', 'Actividad_6816013', 'Actividad_6993019', 'Actividad_6999041', 'Actividad_6999075', 'Actividad_6999108', 'Actividad_7111016', 'Actividad_7112014', 'Actividad_7114010', 'Actividad_7129019', 'Actividad_7212012', 'Actividad_7291016', 'Actividad_7299010', 'Actividad_7511018', 'Actividad_7512016', 'Actividad_7519020', 'Actividad_7611016', 'Actividad_7613004', 'Actividad_7613012', 'Actividad_7613905', 'Actividad_7614010', 'Actividad_8123010', 'Actividad_8123078', 'Actividad_8123094', 'Actividad_8211013', 'Actividad_8219017', 'Actividad_8219122', 'Actividad_8311011', 'Actividad_8312019', 'Actividad_8400002', 'Actividad_8412017', 'Actividad_8415011', 'Actividad_8419013', 'Actividad_8421018', 'Actividad_8424012', 'Actividad_8429038', 'Actividad_8511017', 'Actividad_8519011', 'Actividad_8522014', 'Actividad_8814015', 'Actividad_8824014', 'Actividad_8825012', 'Actividad_8839906', 'Actividad_8915011', 'Actividad_8934029', 'Actividad_8944028', 'Actividad_8944098', 'Actividad_8991011', 'Actividad_9119018', 'Actividad_9319014', 'Actividad_9321019', 'Actividad_9411018', 'Actividad_9471012', 'Actividad_9800100', 'Actividad_9900003', 'Actividad_9999999']]
X_normalized.shape

(29991, 186)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train,X_test,Y_train,Y_test=train_test_split(X_normalized,Y,test_size=.2)

## Regresion Logistica

In [15]:
from sklearn.linear_model import LogisticRegression

lr=LogisticRegression()

lr.fit(X_train,Y_train)

scorel=lr.score(X_test,Y_test)

predictl=lr.predict(X_test)

## Forest

In [16]:
%%time

from sklearn.ensemble import RandomForestClassifier

forest=RandomForestClassifier(n_jobs=2,n_estimators=400)

forest.fit(X_train,Y_train)

scoref=forest.score(X_test,Y_test)

predictf=forest.predict(X_test)

/home/jess/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


CPU times: user 20 s, sys: 145 ms, total: 20.1 s
Wall time: 10.7 s


## SVM

%%time

from sklearn.svm import SVC

C=1e3
svc_rbf = SVC(kernel="rbf", C=C, gamma=0.1) #kernel radial

model= svc_rbf.fit(X_train,Y_train)

scores=model.score(X_test,Y_test)

predicts=model.predict(X_test)

## KNN

%%time

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

knn.fit(X_train, Y_train)

scorek=knn.score(X_test,Y_test)

predictknn=knn.predict(X_test)

#predictknn2=knn.predict_proba(X_test)

## Conclusion

In [18]:
contone=0
contzero=0
total=len(Y_test)

for i in Y_test:
    if(i==1):
        contone=contone+1
    else:
        contzero=contzero+1        
#----------------------------

print('>> LR.... <<')
print('Score',scorel)

c=0
cont=0
cont2=0
for a, b in zip(predictl, Y_test):
    if (a == b):
        c=c+1
        if(a==1):
            cont=cont+1
        else:
            cont2=cont2+1
print(c,'Registros acertados de',total )
print()
#print(pd.crosstab(Y_test,predictl,rownames=['Valor Real'],colnames=['Valor Predicho']))
print('..................')
print()

print('>> Forest.... <<')
print('Score',scoref)

c=0
cont=0
cont2=0
for a, b in zip(predictf, Y_test):
    if (a == b):
        c=c+1
        if(a==1):
            cont=cont+1
        else:
            cont2=cont2+1
print(c,'Registros acertados de',total )
print()
#print(pd.crosstab(Y_test,predictf,rownames=['Valor Real'],colnames=['Valor Predicho']))
print('..................')
print()

>> LR.... <<
Score 0.727621270211702
4365 Registros acertados de 5999

..................

>> Forest.... <<
Score 0.7311218536422737
4386 Registros acertados de 5999

..................



In [44]:
w=forest.predict(X_train)
ww=forest.predict_proba(X_train)

## Obtener mejores variables

In [18]:
%%time

from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
 
rfe=RFECV(lr)
rfe.fit(X_normalized,Y.values.ravel())

CPU times: user 1h 18min 30s, sys: 5min 57s, total: 1h 24min 28s
Wall time: 1h 13min 56s


In [19]:
best_variables=[]

for a, b in zip(X_normalized.columns.values.tolist(), rfe.support_):
    if (b==True):
        best_variables.append(a)
        
print(best_variables)        

['Activo_fijo', 'Macro_sector_', 'Macro_sector_1', 'Macro_sector_2', 'Macro_sector_3', 'Macro_sector_4', 'Macro_sector_5', 'Sector_', 'Sector_1300003', 'Sector_3400009', 'Sector_4100004', 'Sector_6700000', 'Sector_6800002', 'Sector_6900006', 'Sector_7300007', 'Sector_9999999', 'Subsector_', 'Subsector_00000000', 'Subsector_00010000', 'Subsector_0200006-2', 'Subsector_1400001-1', 'Subsector_2300002-1', 'Subsector_2900000-2', 'Subsector_3000007-2', 'Subsector_3400009-2', 'Subsector_3500007-2', 'Subsector_3900009-11', 'Subsector_3900009-6', 'Subsector_6200000-3', 'Subsector_6600002-2', 'Subsector_6600002-5', 'Subsector_6600002-6', 'Subsector_6600002-7', 'Subsector_6800001-1', 'Subsector_6900006-1', 'Subsector_7100001-3', 'Subsector_8100000-3', 'Subsector_8100000-5', 'Subsector_8200008-1', 'Subsector_8200008-4', 'Subsector_8300006-1', 'Subsector_8400004-1', 'Subsector_8400004-2', 'Subsector_8400004-3', 'Subsector_8900004-2', 'Subsector_8900004-5', 'Subsector_9400003-1', 'Subsector_9999999-

In [20]:
rfe.n_features_

186

## Guardar Modelo

In [ ]:
import pickle

filename = 'productos3.sav'
pickle.dump(forest, open(filename, 'wb'))

## Cargar Modelo

In [152]:
import pickle

#filename = 'productos1_lr_avar.sav'
filename = 'productos2_forest_bvar.sav'

model = pickle.load(open(filename, 'rb'))

scorel=model.score(X_test,Y_test)

predictl=model.predict(X_test)


In [153]:
contone=0
contzero=0
total=len(Y_test)

for i in Y_test:
    if(i==1):
        contone=contone+1
    else:
        contzero=contzero+1        
#----------------------------

print('>> Forest.... <<')
print('Score',scorel)

c=0
cont=0
cont2=0
for a, b in zip(predictl, Y_test):
    if (a == b):
        c=c+1
        if(a==1):
            cont=cont+1
        else:
            cont2=cont2+1
print(c,'Registros acertados de',total )
print('..................')

#pd.crosstab(Y_test,predictl,rownames=['Valor Real'],colnames=['Valor Predicho'])


>> Forest.... <<
Score 0.7267877979663278
4360 Registros acertados de 5999
..................
